<a href="https://colab.research.google.com/github/Suvaya/Cryptography/blob/main/Crypto_lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Hill Cipher

In [ ]:
import numpy as np

def create_alphabet_matrix(word):
    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    matrix = []

    for letter in word:
        if letter in alphabet:
            value = ord(letter) - ord('a')
            matrix.append(value)

    # Pad with zeros if needed to have a multiple of 3 elements
    while len(matrix) % 3 != 0:
        matrix.append(0)

    # Reshape the matrix to be 3x2 using column-major order
    num_rows = len(matrix) // 3
    matrix = [matrix[i:i+num_rows] for i in range(0, len(matrix), num_rows)]

    return matrix
#converting my name into a matrix
word = 'suvaya'
matrix = create_alphabet_matrix(word)
# Convert the list of lists to a NumPy array
numpy_array = np.array(matrix)

print(numpy_array)

#conver matrix to alphabets
def matrix_to_alphabets(matrix):
    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    rows, cols = matrix.shape
    result = []

    for i in range(rows):
        row = []
        for j in range(cols):
            letter = alphabet[matrix[i, j]]
            row.append(letter)
        result.append(row)

    return result

#Inverse of matrix driver function
def inverse_2x2_mod(matrix):
    a, b = matrix[0, 0], matrix[0, 1]
    c, d = matrix[1, 0], matrix[1, 1]

    det = (a * d - b * c) % 26

    for i in range(1, 26):
        if (det * i) % 26 == 1:
            det_inv = i
            break

    inv_matrix = np.array([[d, -b], [-c, a]]) * det_inv
    inv_matrix = inv_matrix % 26

    return inv_matrix

#Inverse matrix
matro = np.array([[13, 3], [12, 3]])
inv_matrix = inverse_2x2_mod(matro)
print("Inverse Matrix (mod 26):\n", inv_matrix)

def multiply_matrices(matrix1, matrix2):
    result = np.dot(matrix1, matrix2) % 26
    return result

encrypt_matrix = multiply_matrices(numpy_array, matro)
print(encrypt_matrix)

decrypt_matrix = multiply_matrices(encrypt_matrix, inverse_2x2_mod(matro))
print(decrypt_matrix)

encrypted_texts = matrix_to_alphabets(encrypt_matrix)
decrypted_texts = matrix_to_alphabets(decrypt_matrix)

print(encrypted_texts)
print(decrypted_texts)

[[18 20]
 [21  0]
 [24  0]]
Inverse Matrix (mod 26):
 [[ 1 25]
 [22 13]]
[[ 6 10]
 [13 11]
 [ 0 20]]
[[18 20]
 [21  0]
 [24  0]]
[['g', 'k'], ['n', 'l'], ['a', 'u']]
[['s', 'u'], ['v', 'a'], ['y', 'a']]


#Caesar Cipher

In [ ]:
def ceaserCyEn(plaintext, key):
    encrypt=""
    for char in plaintext:
        if char.isalpha():
            base=ord('A') if char.isupper() else ord('a')

            #find the position of each character
            enc_char = chr((ord(char)-base+key)%26+base)
            encrypt += enc_char
        else:
            encrypt+=char
    return encrypt

def ceaserCyDe(cipherText, key):
    return ceaserCyEn(cipherText, -key)

no = "Suvaya"
key = 5

cipherText = ceaserCyEn(no, key)
print("The ciphertext is: " + cipherText)

print("The deciphered text is: " + ceaserCyDe(cipherText, key))

The ciphertext is: Xzafdf
The deciphered text is: Suvaya


#PlayFair

In [ ]:
extra_letter_pos = []


def generate_playfair_matrix(key):
    key = key.replace("i", "j")  # Replace i with j
    key += "abcdefghijklmnopqrstuvwxyz"

    matrix = [[0] * 5 for _ in range(5)]
    used_letters = set()

    row = 0
    col = 0

    for letter in key:
        if letter not in used_letters:
            matrix[row][col] = letter
            used_letters.add(letter)
            col += 1
            if col == 5:
                col = 0
                row += 1
                if row == 5:
                    break

    return matrix


def find_letter(matrix, letter):
    for i in range(5):
        for j in range(5):
            if matrix[i][j] == letter:
                return i, j


def playfair_encrypt(plaintext, key):
    matrix = generate_playfair_matrix(key)
    plaintext = plaintext.replace("J", "I")
    print(matrix)

    # Add filler 'X' for repeated letters and make pairs
    pairs = []
    i = 0
    while i < len(plaintext):
        if i == len(plaintext) - 1 or plaintext[i] == plaintext[i + 1]:
            pairs.append(plaintext[i] + "X")
            i += 1
            extra_letter_pos.append(i)
        else:
            pairs.append(plaintext[i] + plaintext[i + 1])
            i += 2

    ciphertext = ""

    for pair in pairs:
        row1, col1 = find_letter(matrix, pair[0])
        row2, col2 = find_letter(matrix, pair[1])

        if row1 == row2:
            ciphertext += matrix[row1][(col1 + 1) % 5]
            ciphertext += matrix[row2][(col2 + 1) % 5]
        elif col1 == col2:
            ciphertext += matrix[(row1 + 1) % 5][col1]
            ciphertext += matrix[(row2 + 1) % 5][col2]
        else:
            ciphertext += matrix[row1][col2]
            ciphertext += matrix[row2][col1]

    return ciphertext


def playfair_decrypt(ciphertext, key):
    matrix = generate_playfair_matrix(key)
    ciphertext = ciphertext.replace(" ", "").lower()

    plaintext = ""

    for i in range(0, len(ciphertext), 2):
        row1, col1 = find_letter(matrix, ciphertext[i])
        row2, col2 = find_letter(matrix, ciphertext[i + 1])

        if row1 == row2:
            plaintext += matrix[row1][(col1 - 1) % 5]
            plaintext += matrix[row2][(col2 - 1) % 5]
        elif col1 == col2:
            plaintext += matrix[(row1 - 1) % 5][col1]
            plaintext += matrix[(row2 - 1) % 5][col2]
        else:
            plaintext += matrix[row1][col2]
            plaintext += matrix[row2][col1]

    return plaintext


# Main program
plaintext = input("Enter the plain text: ").replace(" ", "").lower()
key = input("Enter the key: ").replace(" ", "").lower()

encrypted_name = playfair_encrypt(plaintext, key)
print("Encrypted text:", encrypted_name)

decrypted_name = playfair_decrypt(encrypted_name, key)
print("Decrypted text: ", end="")

for char in decrypted_name:
    if decrypted_name.index(char) in extra_letter_pos:
        continue
    print(char, end="")

Enter the plain text: suvaya
Enter the key: help
[['h', 'e', 'l', 'p', 'a'], ['b', 'c', 'd', 'f', 'g'], ['i', 'j', 'k', 'm', 'n'], ['o', 'q', 'r', 's', 't'], ['u', 'v', 'w', 'x', 'y']]
Encrypted text: oxyeag
Decrypted text: suvaya

#Railfence Cipher

In [ ]:
def encrypt_rail_fence(message, rails):
    rail_matrix = [['\n' for _ in range(len(message))] for _ in range(rails)]
    direction = -1  # 1 for moving down, -1 for moving up
    row, col = 0, 0

    for char in message:
        if row == 0 or row == rails - 1:
            direction *= -1
        rail_matrix[row][col] = char
        col += 1
        row += direction

    encrypted_message = ''.join([rail_matrix[i][j] for i in range(rails) for j in range(len(message)) if rail_matrix[i][j] != '\n'])
    return encrypted_message


def decrypt_rail_fence(encrypted_message, rails):
    rail_matrix = [['\n' for _ in range(len(encrypted_message))] for _ in range(rails)]
    direction = -1
    row, col = 0, 0

    for _ in encrypted_message:
        if row == 0 or row == rails - 1:
            direction *= -1
        rail_matrix[row][col] = '*'
        col += 1
        row += direction

    index = 0
    for i in range(rails):
        for j in range(len(encrypted_message)):
            if rail_matrix[i][j] == '*' and index < len(encrypted_message):
                rail_matrix[i][j] = encrypted_message[index]
                index += 1

    direction = -1
    row, col = 0, 0
    decrypted_message = ''

    for _ in encrypted_message:
        if row == 0 or row == rails - 1:
            direction *= -1
        if rail_matrix[row][col] != '\n':
            decrypted_message += rail_matrix[row][col]
        col += 1
        row += direction

    return decrypted_message


message = "suvaya"
rails = 3

encrypted = encrypt_rail_fence(message, rails)
print("Encrypted:", encrypted)

decrypted = decrypt_rail_fence(encrypted, rails)
print("Decrypted:", decrypted)

Encrypted: syuaav
Decrypted: suvaya


#Vignere Cipher

In [ ]:
def generate_vigenere_table():
    alphabet = "abcdefghijklmnopqrstuvwxy"
    vigenere_table = {}

    for i, char in enumerate(alphabet):
        shifted_alphabet = alphabet[i:] + alphabet[:i]
        vigenere_table[char] = shifted_alphabet

    return vigenere_table

def encrypt_vigenere(message, key):
    vigenere_table = generate_vigenere_table()
    encrypted_message = ""

    message = message.lower()
    key = key.lower()
    key_index = 0

    for char in message:
        if char.isalpha():
            key_char = key[key_index % len(key)]
            encrypted_char = vigenere_table[key_char][ord(char) - ord('a')]
            encrypted_message += encrypted_char
            key_index += 1
        else:
            encrypted_message += char

    return encrypted_message

def decrypt_vigenere(encrypted_message, key):
    vigenere_table = generate_vigenere_table()
    decrypted_message = ""

    encrypted_message = encrypted_message.lower()
    key = key.lower()
    key_index = 0

    for char in encrypted_message:
        if char.isalpha():
            key_char = key[key_index % len(key)]
            decrypted_char = chr(vigenere_table[key_char].index(char) + ord('a'))
            decrypted_message += decrypted_char
            key_index += 1
        else:
            decrypted_message += char

    return decrypted_message

message = "suvayagopalshrestha"
key = "ghorlay"

encrypted = encrypt_vigenere(message, key)
print("Encrypted:", encrypted)

decrypted = decrypt_vigenere(encrypted, key)
print("Decrypted:", decrypted)

Encrypted: yckrkafuwodehqkaiyl
Decrypted: suvayagopalshrestha


Encrypted: aaggffhafccffbh
Decrypted: dadcafeeachbabe
